In [1]:
!pip install gurobipy

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import numpy as np
import gurobipy as gp

In [3]:
M = 10000.0
N = 5
distMat = np.array([[M,132,217,164,58],[132,M,290,201,79],[217,290,M,113,303],[164,201,113,M,196],[58,79,303,196,M]])



In [4]:
mod = gp.Model()
x = mod.addMVar((N,N),vtype='B')
u = mod.addMVar(N)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19


In [5]:
mod.setObjective(gp.quicksum(x[i,j]*distMat[i,j] for i in range(N) for j in range(N)))

In [6]:

# leave each city once
leave = mod.addConstrs((gp.quicksum(x[i,j] for i in range(N))==1) for j in range(N))
# enter each city once
enter = mod.addConstrs((gp.quicksum(x[i,j] for j in range(N))==1) for i in range(N))
# u constraints
ucons = mod.addConstrs((u[i] - u[j] + N*x[i,j] <= (N-1)) for i in range(1,N) for j in range(1,N) if i != j)

In [7]:
len(ucons)

12

In [8]:
(N-1)**2 - (N-1)

12

In [9]:
mod.Params.OutputFlag = 0 # tell gurobi to shut up!!
mod.optimize()

In [10]:
mod.objVal

668.0

In [11]:
where = 0
miles_traveled = 0
print('Start in city 0.')
for city in range(N):
    where_new = np.where(x.x[where,:])[0][0]
    dist = distMat[where,where_new]
    miles_traveled += dist
    print('Then go to city '+str(where_new)+', traveling '+str(dist)+' miles along the way.')
    where = where_new
print('For a total of '+str(miles_traveled)+' miles.')

Start in city 0.
Then go to city 2, traveling 217.0 miles along the way.
Then go to city 3, traveling 113.0 miles along the way.
Then go to city 1, traveling 201.0 miles along the way.
Then go to city 4, traveling 79.0 miles along the way.
Then go to city 0, traveling 58.0 miles along the way.
For a total of 668.0 miles.


In [12]:
# this path may not be unique, there are other paths that are optimal
# but they'll all have the same total miles traveled!
# for example 0 -> 2 -> 3 -> 1 -> 4 -> 0
# also takes 668 miles!

In [13]:
seq = [0, 2, 3, 1, 4, 0]
miles_traveled2 = 0
for loc in range(N):
    where = seq[loc]
    where_new = seq[loc+1]
    dist = distMat[where,where_new]
    miles_traveled2 += dist
print(miles_traveled2)

668.0


In [14]:
# I have no idea how many other paths are optimal...